In [1]:
import re
import time
import json
import csv
import random
import requests
import pprint
import pickle
import unidecode
import numpy as np
import pandas as pd
from random import sample
import reverse_geocode
import seaborn as sns
from sklearn import metrics
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import chisquare
from collections import Counter, defaultdict
from sklearn.metrics import cohen_kappa_score

In [2]:
Data_Root = '/Data/Promotion/'

In [3]:
def yield_one_line(filename, delimiter = '\t', quote = csv.QUOTE_NONE):
    '''a generator which produce one line of a given file'''
    with open(filename, 'r') as file:
        print('processing %s...' %(filename))
        reader = csv.reader(file, delimiter=delimiter, quoting=quote)
        for row in reader:
            yield row

In [4]:
plat_cate = {
 'blogs': 'blog',
 'news': 'news_media',
 'patent': 'knowledge_repo',
 'f1000': 'knowledge_repo',
 'peer_reviews': 'knowledge_repo',
 'wikipedia': 'knowledge_repo',
 'q&a': 'knowledge_repo',
 'facebook': 'social_media',
 'googleplus': 'social_media',
 'linkedin': 'social_media',
 'pinterest': 'social_media',
 'reddit': 'social_media',
 'twitter': 'social_media',
 'video': 'social_media'
}

In [5]:
years = set(['2013', '2014', '2015', '2016', '2017', '2018'])

In [ ]:
paper_li = {}
cn_lines = 0

with open(Data_Root + 'merged.txt', 'r') as ofile:
    for row in ofile:
        cn_lines += 1
        if cn_lines % 1000000 == 0:
            print('processed %d lines...'%cn_lines)
        if '}{"altmetric_id"' in row:
            row = row.replace('}{"altmetric_id"', '}\n{"altmetric_id"')
        # this way works for both good and bad lines.
        records = row.split('\n')
        for record in records:
            if record != '':
                paper = json.loads(record)
                citation = paper['citation']
                pubdate = ''
                if 'pubdate' in citation:
                    pubdate = citation['pubdate']
                elif 'epubdate' in citation:
                    pubdate = citation['epubdate']
                dtype = ''
                if 'type' in citation:
                    dtype = citation['type']
                doi = ''
                if 'doi' in citation:
                    doi = citation['doi']
                # focus on research articles published in [2013, 2018]
                if pubdate != '' and pubdate[:4] in years and dtype == 'article' and doi != '':
                    # authors = []
                    # if 'authors' in citation:
                    #    authors = citation['authors']
                    doi = doi.lower()
                    jnl = ''
                    if 'journal' in citation:
                        jnl = citation['journal']
                    scopus_subjects = []
                    if 'scopus_subjects' in citation:
                        scopus_subjects = citation['scopus_subjects']
                    tweets = []
                    if 'twitter' in paper['posts']:
                        tweets = paper['posts']['twitter']
                    # could add pub year later here.
                    pp = {'journal': jnl, 'year': pubdate[:4], 'pub_date':pubdate, 'scopus_subjects': scopus_subjects, 'tweets': tweets}
                    paper_li[doi] = pp

processed 1000000 lines...
processed 2000000 lines...
processed 3000000 lines...
processed 4000000 lines...
processed 5000000 lines...
processed 6000000 lines...
processed 7000000 lines...
processed 8000000 lines...
processed 9000000 lines...
processed 10000000 lines...
processed 11000000 lines...
processed 12000000 lines...
processed 13000000 lines...
processed 14000000 lines...
processed 15000000 lines...
processed 16000000 lines...
processed 17000000 lines...
processed 18000000 lines...
processed 19000000 lines...
processed 20000000 lines...
processed 21000000 lines...
processed 22000000 lines...
processed 23000000 lines...
processed 24000000 lines...


In [12]:
len(paper_li)

6601528

In [11]:
with open(Data_Root+'revision/papers_2013_2018.json', 'w') as ofile:
    for doi in paper_li:
        res = {'doi': doi, 'data': paper_li[doi]}
        ofile.write(json.dumps(res)+'\n')

In [6]:
paper_li = {}
with open(Data_Root+'revision/papers_2013_2018.json', 'r') as ifile:
    for line in ifile:
        res = json.loads(line)
        paper_li[res['doi']] = res['data']

In [8]:
year_cn = defaultdict(int)

for doi in paper_li:
    paper = paper_li[doi]
    year_cn[paper['year']] += 1

In [11]:
year_cn

defaultdict(int,
            {'2013': 875371,
             '2016': 1187604,
             '2014': 981578,
             '2017': 1232949,
             '2015': 1105317,
             '2018': 1218709})

In [16]:
discs_broad = set(['Social Sciences', 'Life Sciences', 'Health Sciences', 'Physical Sciences', 'General'])

# every paper has one of the five broad disciplines.
# every paper has subfiels and their associated broad disciplines, so there is no need to map sub to broad separately.
i = 0
for doi in paper_li:
    paper = paper_li[doi]
    if len(paper['scopus_subjects']) > 0:
        for sub in paper['scopus_subjects']:
            if sub in discs_broad:
                break
        else:
            if i < 10:
                print(paper['scopus_subjects'])
            i += 1

In [13]:
# ['General']
# ['Physical Sciences', 'General', 'Mathematics', 'Engineering']
sub_cn = defaultdict(int)
for doi in paper_li:
    paper = paper_li[doi]
    for sub in paper['scopus_subjects']:
        sub_cn[sub] += 1

In [14]:
# num. of papers with a specific keyword
sorted(sub_cn.items(), key=lambda x: x[1]*-1)

[('Health Sciences', 2176418),
 ('Medicine', 2059458),
 ('Life Sciences', 1536250),
 ('Physical Sciences', 1448583),
 ('Biochemistry, Genetics and Molecular Biology', 912785),
 ('Social Sciences', 659042),
 ('Agricultural and Biological Sciences', 478068),
 ('Physics and Astronomy', 419434),
 ('Chemistry', 394765),
 ('Engineering', 286386),
 ('Environmental Science', 258819),
 ('Materials Science', 242681),
 ('Earth and Planetary Sciences', 207029),
 ('Neuroscience', 204842),
 ('Immunology and Microbiology', 192735),
 ('Psychology', 182761),
 ('Pharmacology, Toxicology and Pharmaceutics', 172871),
 ('Chemical Engineering', 161673),
 ('Computer Science', 155894),
 ('Mathematics', 137190),
 ('Nursing', 119965),
 ('Business, Management and Accounting', 89325),
 ('Arts and Humanities', 87608),
 ('Health Professions', 79649),
 ('Economics, Econometrics and Finance', 73336),
 ('General', 62859),
 ('Energy', 61888),
 ('Veterinary', 45343),
 ('Decision Sciences', 34259),
 ('Dentistry', 26390)]

In [15]:
len(sub_cn)

30

In [16]:
# Social Sciecnes is created in the subfields because there is also a `social sciences` subfield category.
discs = {'Life Sciences': 0, 'Health Sciences': 1, 'Physical Sciences': 2}

# 26 subfields + 1 General
fields = {'Social Sciences': 'Social Sciences',
 'Materials Science': 'Physical Sciences',
 'Engineering': 'Physical Sciences',
 'Chemistry': 'Physical Sciences',
 'Biochemistry, Genetics and Molecular Biology': 'Life Sciences',
 'Medicine': 'Health Sciences',
 'Nursing': 'Health Sciences',
 'Agricultural and Biological Sciences': 'Life Sciences',
 'Pharmacology, Toxicology and Pharmaceutics': 'Life Sciences',
 'Neuroscience': 'Life Sciences',
 'Business, Management and Accounting': 'Social Sciences',
 'Economics, Econometrics and Finance': 'Social Sciences',
 'Chemical Engineering': 'Physical Sciences',
 'Physics and Astronomy': 'Physical Sciences',
 'Computer Science': 'Physical Sciences',
 'Decision Sciences': 'Social Sciences',
 'Health Professions': 'Health Sciences',
 'Psychology': 'Social Sciences',
 'Immunology and Microbiology': 'Life Sciences',
 'Dentistry': 'Health Sciences',
 'Earth and Planetary Sciences': 'Physical Sciences',
 'Environmental Science': 'Physical Sciences',
 'Mathematics': 'Physical Sciences',
 'Arts and Humanities': 'Social Sciences',
 'Energy': 'Physical Sciences',
 'Veterinary': 'Health Sciences',
 'General': 'General'}

field_ix = {fie: ix for ix, fie in enumerate(list(fields.keys()))}

In [17]:
len(fields)

27

In [18]:
with open(Data_Root+'revision/alt_dois.pickle', 'wb') as ofile:
    pickle.dump(list(paper_li.keys()), ofile)

In [25]:
paper_li[doi]

{'journal': 'Criminology & Public Policy',
 'scopus_subjects': [],
 'tweets': [{'license': 'gnip',
   'citation_ids': [46502633],
   'author': {'tweeter_id': '3249180837'},
   'tweet_id': '1029346936497823745'},
  {'license': 'gnip',
   'citation_ids': [46502633],
   'author': {'tweeter_id': '3257735117'},
   'tweet_id': '1032335832299040769'},
  {'license': 'gnip',
   'citation_ids': [46502633],
   'author': {'tweeter_id': '987647622105194502'},
   'tweet_id': '1032344553259524096'},
  {'license': 'gnip',
   'citation_ids': [46502633],
   'author': {'tweeter_id': '19022052'},
   'tweet_id': '1036613785598021632'}]}

In [ ]:
# tid_name_sname_org = {}
# tid_name_sname_retweet = {}

# with open(Data_Root+'tweets_v2.json', 'r') as ifile:
#     for line in ifile:
#         line = json.loads(line)
#         tid = line['id_str'] # line['created_at']
#         if 'retweeted_status' in line:
#             tid_name_sname_retweet[tid] = (line['user']['name'], line['user']['screen_name'])
#         else:
#             # pprint.pprint(line)
#             tid_name_sname_org[tid] = (line['user']['name'], line['user']['screen_name'])
#             # tid_date_all[tid] = datetime.strptime(tdate, '%a %b %d %H:%M:%S +0000 %Y')#.strftime("%Y-%m-%d")
        
# with open(Data_Root+'tweets_v2_non_full_text.json', 'r') as ifile:
#     for line in ifile:
#         line = json.loads(line)
#         tid = line['id_str']
#         if 'retweeted_status' in line:
#             tid_name_sname_retweet[tid] = (line['user']['name'], line['user']['screen_name'])
#         else:
#             tid_name_sname_org[tid] = (line['user']['name'], line['user']['screen_name'])

In [ ]:
# 85411606 tids in total, which contains 14412 tids that are not numeric string.
# e.g., '1.5558470649316E+17', '1.5464891076012E+17', '1.6227338710274E+17'

In [26]:
# An example tweet structure

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Feb 17 09:01:16 +0000 2017',
 'display_text_range': [0, 116],
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'bit.ly/2kqg0ym',
                        'expanded_url': 'http://bit.ly/2kqg0ym',
                        'indices': [77, 100],
                        'url': 'https://t.co/PcpdHjw86p'}],
              'user_mentions': [{'id': 96741105,
                                 'id_str': '96741105',
                                 'indices': [0, 15],
                                 'name': 'Newcastle University',
                                 'screen_name': 'UniofNewcastle'},
                                {'id': 96741105,
                                 'id_str': '96741105',
                                 'indices': [101, 116],
                                 'name': 'Newcastle University',
                                 'screen_name': 'UniofNewcastle'}]},
 'f

In [67]:
with open(Data_Root+'tid_name_sname.json', 'w') as ofile:
    for tid in tid_name_sname_org:
        res = {'tid': tid, 'type': 'org', 'data': tid_name_sname_org[tid]}
        ofile.write(json.dumps(res)+'\n')
    for tid in tid_name_sname_retweet:
        res = {'tid': tid, 'type': 're', 'data': tid_name_sname_retweet[tid]}
        ofile.write(json.dumps(res)+'\n')

In [20]:
tid_name_sname_org = {}
tid_name_sname_retweet = {}

with open(Data_Root+'tid_name_sname.json', 'r') as ifile:
    for line in ifile:
        res = json.loads(line)
        if res['type'] == 'org':
            tid_name_sname_org[res['tid']] = res['data']
        else:
            tid_name_sname_retweet[res['tid']] = res['data']

In [21]:
len(tid_name_sname_org)

34362660

In [22]:
len(tid_name_sname_retweet)

43134644

In [16]:
doi_author_list = defaultdict(list)

with open(Data_Root+'revision/dois_authors_mag.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        doi, authors = row['doi'], row['authors']
        uniq_authors = defaultdict(lambda: ['', [], ''])
        for aid, affi_id, seq, name in authors:
            seq = int(seq)
            uniq_authors[seq][0] = aid
            uniq_authors[seq][1].append(affi_id)
            # normalized author name.
            uniq_authors[seq][2] = unidecode.unidecode(name).strip()
        # could be [1, 2, 4, 6]; [2, 3]
        uniq_seqs = sorted(uniq_authors)
        # must be: 1, 2, ..., max
        if uniq_seqs == list(range(1, len(uniq_seqs)+1)):
            last_seq = len(uniq_seqs)
            if last_seq == 1:
                aid, affi_ids, name = uniq_authors[last_seq]
                affis = '|'.join(affi_ids)
                doi_author_list[doi].append([aid, affis, last_seq, name, 'solo_author'])
            else:
                for seq in uniq_authors:
                    aid, affi_ids, name = uniq_authors[seq]
                    affis = '|'.join(affi_ids)
                    if seq == 1:
                        doi_author_list[doi].append([aid, affis, seq, name, 'first_position'])
                    elif seq == last_seq:
                        doi_author_list[doi].append([aid, affis, seq, name, 'last_position'])
                    else:
                        doi_author_list[doi].append([aid, affis, seq, name, 'middle_position'])

In [17]:
len(doi_author_list)

6061830

In [18]:
doi_author_list['10.1001/amajethics.2018.1003']

[['2899491243', '', 1, 'Annie J. Tsay', 'solo_author']]

In [31]:
author_names = set()
for doi in doi_author_list:
    for aut in doi_author_list[doi]:
        author_name = aut[3]
        author_names.add(author_name)

In [32]:
len(author_names)

8507860

In [33]:
with open(Data_Root+'revision/author_names.txt', 'w') as ofile:
    for author_name in author_names:
        ofile.write(author_name+'\n')

In [19]:
# predict gender based on name using Ford et al.
# see `1_3_predict_gender.ipynb` (run code using Python 2.7)

name_gender = {}
with open(Data_Root+'revision/name_gender.txt', 'r') as ifile:
    for line in ifile:
        # get rid of '\n' at the end of each line
        line = line.strip()
        name, gender = line.split('||')
        name_gender[name] = gender

In [20]:
len(name_gender)

8454095

In [21]:
# see `1_2_mag.ipynb`

aid_metric = {}

with open(Data_Root+'revision/aids_metric_mag.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        aid, metric = row['aid'], row['metric']
        aid_metric[aid] = [int(num) for num in metric]

In [22]:
len(aid_metric)

9777945

In [49]:
aid_year_paper_count = {}
with open(Data_Root+'revision/aid_year_paper_list.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        aid = row['aid']
        aid_year_paper_count[aid] = {year: len(pids) for year, pids in row['year_pids'].items()}

In [50]:
len(aid_year_paper_count)

9776277

In [51]:
aid_year_paper_count['2704166667']

{'2015': 7,
 '2010': 2,
 '2013': 1,
 '2011': 1,
 '2014': 4,
 '2008': 1,
 '2005': 2,
 '2003': 1,
 '2012': 3,
 '2016': 8,
 '2017': 7,
 '2018': 16}

In [7]:
# see `1_2_mag.ipynb`

aid_cites_upto_years = {}

with open(Data_Root+'revision/aid_cites_upto_years.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        aid, cites = row['aid'], row['cum_cites']
        aid_cites_upto_years[aid] = cites

In [27]:
len(aid_cites_upto_years)

8744877

In [28]:
aid_cites_upto_years['2704166667']

{'2015': 285, '2016': 415, '2017': 598, '2012': 120, '2013': 146, '2014': 200}

In [29]:
def get_author_name(name):
    words = name.split()
    if len(words) <= 1:
        payload = 'invalid'
    else:
        payload = {'Fname': words[0], 'Lname': words[-1]}
    return payload

def check_match_author(tname_tuple, name_tupe):
    hname, sname = tname_tuple
    hname = unidecode.unidecode(hname).lower()
    sname = unidecode.unidecode(sname).lower()
    htokens, stokens = [hname], [sname]
    if ' ' in hname:
        htokens = hname.split(' ')
    elif '_' in hname:
        htokens = hname.split('_')
    if ' ' in sname:
        stokens = sname.split(' ')
    elif '_' in sname:
        stokens = sname.split('_')
    tokens = htokens + stokens
    fname, lname = name_tupe['Fname'], name_tupe['Lname']
    fname = fname.lower()
    lname = lname.lower()
    if len(tokens) > 2:
        # don't use first name if it is initial. e.g., True 3 M. B. Mendonca (Plecoforum.nl)
        if (len(fname) == 1) or (len(fname) == 2 and fname[-1] == '.'):
            if lname in tokens:
                return True
        else:
            if (fname in tokens) or (lname in tokens):
                return True
        return False
    # both user name and screen name are single-token string: use containment matching only for names with >= 4 chars.
    else:
        if len(fname) >= 4:
            if (fname in hname) or (fname in sname):
                return True
        if len(lname) >= 4:
            if (lname in hname) or (lname in sname):
                return True
        return False

### Build regression data

In [18]:
table = str.maketrans(",'&:-", "     ")

def norm_string(aff_name):
    aff_name = aff_name.lower().translate(table)
    aff_name = ' '.join(aff_name.split())
    return aff_name

def get_journal_rank(journal):
    if journal == 'unknown':
        return np.NaN
    else:
        nname = norm_string(journal)
        if nname in journal_impact:
            return journal_impact[nname]
        else:
            return np.NaN
        
def get_journal_cate(journal):
    if journal in top_journals or journal == 'unknown':
        return journal
    else:
        return 'lose journal'

def get_affi_name(affis):
    affi_li = affis.split('|')
    univs = []
    for affi_id in affi_li:
        if affi_id in affi_name:
            univs.append(affi_name[affi_id])
    if len(univs) == 0:
        return 'unknown'
    else:
        return '|'.join(univs)
    
def get_affi_cate(affis):
    affi_li = affis.split('|')
    cate_li = set()
    for affi_id in affi_li:
        # many affi ids in MAG do not have latitude and longitude info for us to infer their country
        if affi_id in affi_country:
            country = affi_country[affi_id]
            if country == 'United States':
                cate_li.add('domestic')
            else:
                cate_li.add('international')
        else:
            cate_li.add('unknown')
    if 'domestic' in cate_li:
        return 'domestic'
    elif 'international' in cate_li:
        return 'international'
    else:
        return 'unknown'
        
def get_affi_rank(affis):
    affi_li = affis.split('|')
    rank_li = []
    for affi_id in affi_li:
        if affi_id in affi_rank:
            rank_li.append(affi_rank[affi_id])
    if len(rank_li) == 0:
        return np.NaN
    else:
        return min(rank_li)

def get_author_rank(aid):
    if aid in aid_metric:
        return aid_metric[aid][0]
    else:
        return np.NaN    

def get_aut_pub_cn(row):
    pub_year, aid = row['pub_year'], row['author_id']
    if aid in aid_year_paper_count:
        total = 0
        pub_year = int(pub_year)
        for year, cn in aid_year_paper_count[aid].items():
            if int(year) <= pub_year:
                total += cn
        return total
    else:
        return np.NaN

def get_aut_cite_cn(row):
    pub_year, aid = row['pub_year'], row['author_id']
    if aid in aid_cites_upto_years:
        ykey = str(int(pub_year) - 1)
        if ykey in aid_cites_upto_years[aid]:
            return aid_cites_upto_years[aid][ykey]
        else:
            return 0
    else:
        return np.NaN
    
def get_field_vector(doi):
    vec = [0] * len(fields)
    for field in paper_li[doi]['scopus_subjects']:
        if field in field_ix:
            vec[field_ix[field]] = 1
    return vec

def get_disc_vector(doi):
    vec = [0] * len(discs)
    for field in paper_li[doi]['scopus_subjects']:
        if field in discs:
            vec[discs[field]] = 1
    return vec

def clean_field_name(name):
    ans = ''
    for ch in name:
        if ch in ['(', ',', ')', '-', ' ', "'"]:
            ans += '_'
        else:
            ans += ch
    return ans

# make `tid_name_sname` a param.
def get_self_promo(row, tid_name_sname):
    doi, author_name = row['doi'], row['author_name']
    paper = paper_li[doi]
    mtids = [tweet['tweet_id'] for tweet in paper['tweets'] if tweet['tweet_id'] in tid_name_sname]
    # must already be a dict.
    name_tupe = get_author_name(author_name)
    self_promo = False
    mat_tid = ''
    matches = []
    for tid in mtids:
        tname_tupe = tid_name_sname[tid]
        if check_match_author(tname_tupe, name_tupe):
            matches.append((tid, tid_name_sname[tid][0]))
    if len(matches) >= 1:
        self_promo = True
        if len(matches) == 1:
            mat_tid = matches[0][0]
        else:
            tid_scores = {}
            for tid, tw_name in matches:
                tid_scores[tid] = fuzz.ratio(tw_name, author_name)
            mat_tid = sorted(tid_scores.keys(), key=lambda tid: tid_scores[tid], reverse=True)[0]
    return (self_promo, mat_tid)

def get_self_promo_comb_tid(row):
    p_0, p_1, p_2 = row['self_promotion'], row['self_promotion_original'], row['self_promotion_retweet']
    if p_0:
        if p_1:
            return row['matched_tid_original']
        # p_2 must be true
        else:
            return row['matched_tid_retweet']
    else:
        return ''

In [57]:
reg_data = []

cn = 0
for doi in paper_li:
    cn += 1
    if cn%100000 == 0:
        print('processed %d papers...'%cn)
    pub_year = paper_li[doi]['year']
    if doi in doi_author_list:
        # get pub year from paper_li later
        for author in doi_author_list[doi]:
            aid, affis, seq, author_name, pos = author
            name_tupe = get_author_name(author_name)
            if type(name_tupe) == dict:
#                 self_promo, mat = get_self_promo(doi, author_name)
                reg_data.append([doi, pub_year, author_name, seq, pos, aid, affis])

processed 100000 papers...
processed 200000 papers...
processed 300000 papers...
processed 400000 papers...
processed 500000 papers...
processed 600000 papers...
processed 700000 papers...
processed 800000 papers...
processed 900000 papers...
processed 1000000 papers...
processed 1100000 papers...
processed 1200000 papers...
processed 1300000 papers...
processed 1400000 papers...
processed 1500000 papers...
processed 1600000 papers...
processed 1700000 papers...
processed 1800000 papers...
processed 1900000 papers...
processed 2000000 papers...
processed 2100000 papers...
processed 2200000 papers...
processed 2300000 papers...
processed 2400000 papers...
processed 2500000 papers...
processed 2600000 papers...
processed 2700000 papers...
processed 2800000 papers...
processed 2900000 papers...
processed 3000000 papers...
processed 3100000 papers...
processed 3200000 papers...
processed 3300000 papers...
processed 3400000 papers...
processed 3500000 papers...
processed 3600000 papers...
p

In [58]:
reg_data = pd.DataFrame(reg_data, columns = ['doi', 'pub_year', 'author_name', 'authorship_seq', 'authorship_pos', \
                                             'author_id', 'affiliation_ids'])

In [59]:
reg_data['DV'] = reg_data.apply(lambda row: get_self_promo(row, tid_name_sname_org), axis = 1)
reg_data[['self_promotion_original', 'matched_tid_original']] = pd.DataFrame(reg_data.DV.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['DV'])


In [60]:
np.sum(reg_data['self_promotion_original'] == True)

740636

In [61]:
# use a different `tid_name_sname` for retweets.
reg_data['DV'] = reg_data.apply(lambda row: get_self_promo(row, tid_name_sname_retweet), axis = 1)
reg_data[['self_promotion_retweet', 'matched_tid_retweet']] = pd.DataFrame(reg_data.DV.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['DV'])


In [62]:
np.sum(reg_data['self_promotion_retweet'] == True)

515462

In [64]:
reg_data['self_promotion'] = (reg_data['self_promotion_retweet'] == True) | (reg_data['self_promotion_original'] == True)
reg_data['matched_tid'] = reg_data.apply(lambda row: get_self_promo_comb_tid(row), axis = 1)


In [65]:
np.sum(reg_data['self_promotion'] == True)

1061401

In [ ]:
# Altmetric data: JAMA: Journal of the American Medical Association
# WOS data: JAMA-JOURNAL OF THE AMERICAN MEDICAL ASSOCIATION
# 2018 version.
journal_impact = {}
for line in yield_one_line(Data_Root+'wos_jcr.csv', delimiter=',', quote=csv.QUOTE_ALL):
    rank, title, cites, jif, eigen = line
    try:
        impact = float(jif)
        ntitle = norm_string(title)
        journal_impact[ntitle] = impact
    except:
        continue

In [43]:
len(journal_impact)

11873

In [ ]:
affi_rank = {}
affi_country = {}
affi_name = {}
for line in yield_one_line(Data_Root+'Affiliations.txt'):
    affi_id, rank, dname, lat, lon = line[0], line[1], line[3], line[9], line[10]
    affi_rank[affi_id] = int(rank)
    affi_name[affi_id] = dname
    if lat != "" and lon != "":
        lat, lon = float(lat), float(lon)
        res = reverse_geocode.search([(lat, lon)])
        country = res[0]['country']
        affi_country[affi_id] = country

In [45]:
# affi_name

Controls

In [67]:
reg_data['gender'] = reg_data['author_name'].apply(lambda author_name: name_gender[author_name])

In [68]:
reg_data['num_authors'] = reg_data['doi'].apply(lambda doi: len(doi_author_list[doi]))

In [69]:
reg_data['author_pub_count'] = reg_data.apply(lambda row: get_aut_pub_cn(row), axis = 1)

In [19]:
reg_data['author_citation'] = reg_data.apply(lambda row: get_aut_cite_cn(row), axis = 1)

In [80]:
# reg_data['author_feats'] = reg_data['author_id'].map(aid_metric)
# reg_data[['author_rank', 'num_paper', 'num_cite']] = pd.DataFrame(reg_data['author_feats'].values.tolist(), index = reg_data.index)

reg_data['author_rank'] = reg_data['author_id'].apply(get_author_rank)
reg_data['affiliation_name'] = reg_data['affiliation_ids'].apply(get_affi_name)
reg_data['affiliation_cate'] = reg_data['affiliation_ids'].apply(get_affi_cate)
reg_data['affiliation_rank'] = reg_data['affiliation_ids'].apply(get_affi_rank)
reg_data['journal_title'] = reg_data['doi'].apply(lambda doi: paper_li[doi]['journal'] if paper_li[doi]['journal'] != '' else 'unknown')
reg_data['journal_impact'] = reg_data['journal_title'].apply(get_journal_rank)
top_journals = set([jname for jname, num in Counter(reg_data['journal_title']).most_common()[:100] if jname != 'unknown'])
reg_data['top_journal'] = reg_data['journal_title'].apply(get_journal_cate)

In [81]:
aid_rate_pub_cn = {}
for aid, sub in reg_data.groupby('author_id'):
    pub_cn = len(sub)
    aid_rate_pub_cn[aid] = (pub_cn, np.mean(sub['self_promotion']))

In [82]:
reg_data['pub_cn_rate'] = reg_data['author_id'].map(aid_rate_pub_cn)
reg_data[['author_num_papers_in_data', 'author_self_promotion_rate']] = pd.DataFrame(reg_data.pub_cn_rate.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['pub_cn_rate'])

In [83]:
del aid_rate_pub_cn

Discipline

In [84]:
reg_data['field_vector'] = reg_data['doi'].apply(get_field_vector)
reg_data[list(fields.keys())] = pd.DataFrame(reg_data.field_vector.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['field_vector'])

In [85]:
field_name_clean_map = {name: clean_field_name(name) for name in fields}

In [86]:
' + '.join(field_name_clean_map.values())

'Social_Sciences + Materials_Science + Engineering + Chemistry + Biochemistry__Genetics_and_Molecular_Biology + Medicine + Nursing + Agricultural_and_Biological_Sciences + Pharmacology__Toxicology_and_Pharmaceutics + Neuroscience + Business__Management_and_Accounting + Economics__Econometrics_and_Finance + Chemical_Engineering + Physics_and_Astronomy + Computer_Science + Decision_Sciences + Health_Professions + Psychology + Immunology_and_Microbiology + Dentistry + Earth_and_Planetary_Sciences + Environmental_Science + Mathematics + Arts_and_Humanities + Energy + Veterinary + General'

In [87]:
reg_data = reg_data.rename(columns=field_name_clean_map)

In [88]:
reg_data['disc_vector'] = reg_data['doi'].apply(get_disc_vector)
reg_data[list(discs.keys())] = pd.DataFrame(reg_data.disc_vector.values.tolist(), index = reg_data.index)
reg_data = reg_data.drop(columns=['disc_vector'])

In [89]:
disc_name_clean_map = {name: clean_field_name(name) for name in discs}

In [90]:
reg_data = reg_data.rename(columns=disc_name_clean_map)

Group affiliations

In [91]:
reg_data.affiliation_rank.describe()

count    2.132341e+07
mean     7.565661e+03
std      1.749812e+03
min      4.446000e+03
25%      6.308000e+03
50%      7.255000e+03
75%      8.531000e+03
max      1.923800e+04
Name: affiliation_rank, dtype: float64

In [92]:
np.sum(reg_data.affiliation_rank.isna())

10632779

In [93]:
reg_data['affiliation_rank_cate'] = pd.qcut(reg_data['affiliation_rank'], q=10, labels=False, duplicates='drop')

In [94]:
reg_data.affiliation_rank_cate.value_counts()

5.0    2156287
2.0    2142618
1.0    2142223
0.0    2139495
8.0    2133045
9.0    2130291
7.0    2122499
4.0    2121821
6.0    2117948
3.0    2117179
Name: affiliation_rank_cate, dtype: int64

In [95]:
# NaN values are ignored (no cate assigned)
np.sum(reg_data.affiliation_rank_cate.value_counts())

21323406

Group num. of pubs

In [96]:
reg_data.author_pub_count.describe()

count    3.195454e+07
mean     8.909679e+01
std      1.919776e+02
min      0.000000e+00
25%      4.000000e+00
50%      2.300000e+01
75%      8.900000e+01
max      4.871000e+03
Name: author_pub_count, dtype: float64

They added an option duplicates='raise'|'drop' to control whether to raise on duplicated edges or to drop them, which would result in less bins than specified, and some larger (with more elements) than others.

In [97]:
out, bins = pd.qcut(reg_data['author_pub_count'], q=10, labels=False, duplicates='drop', retbins=True)

In [98]:
np.set_printoptions(suppress=True)
bins

array([   0.,    1.,    3.,    6.,   13.,   23.,   41.,   69.,  118.,
        229., 4871.])

In [99]:
reg_data['author_pub_count_cate'] = pd.qcut(reg_data['author_pub_count'], q=10, labels=False, duplicates='drop')


In [100]:
reg_data.author_pub_count_cate.value_counts()

0.0    4064583
3.0    3427521
5.0    3328370
9.0    3194847
8.0    3169567
6.0    3147448
7.0    3135451
1.0    3013007
4.0    2951678
2.0    2522065
Name: author_pub_count_cate, dtype: int64

In [101]:
reg_data.shape

(31956185, 58)

In [102]:
reg_data.to_csv(Data_Root+"revision/reg_data.csv", index=False, header=True, encoding='utf-8')

In [42]:
utype = {'author_id': str, 'matched_tid': 'str', 'matched_tid_original': 'str', 'matched_tid_retweet': 'str'}
reg_data = pd.read_csv(Data_Root+"revision/reg_data.csv", header=0, dtype=utype)